In [1]:
import random
import re, string,os
from glob import glob as gb
import pandas as pd
from collections import Counter
from tqdm import tqdm
from datetime import datetime, timedelta, date
from collections import OrderedDict
import subprocess
import matplotlib.pyplot as plt
import seaborn as sns
import math
from functions import *
import nltk

In [3]:
tf = pd.read_csv(base_path + '/resources/keywords-expertise-translation.csv')
tf_science = {language:dict(zip(tf[tf['language'] == language]['word'],tf[tf['language'] == language]['translation'])) for language in list(set(tf['language']))}
tf_science = {k:{x:i for x,i in v.items() if x not in ["policy","program","measures"]} for k,v in tf_science.items()}

In [30]:
def get_pmi_score(dfw,word1,word2,date):
    data_path = "~/Documents/GitHub/ParlaMintCase/data/original/pl/pl-ana-txt"
    f1 = len(dfw[dfw.id.str.split('_').str[1].str[:7] == date])
    p1 = f1 / totals[date]
    f2 = int(subprocess.check_output(f'egrep -iE "{word2}"  {data_path}/*{date}* | wc -l',shell=True).decode('utf-8'))  
    p2 = f2 / totals[date]
    f12 = len(dfw[(dfw.id.str.split('_').str[1].str[:7] == date) & (dfw.window.str.contains(word2))])  
    p12 = f12 / totals[date]
    if p12 > 0:
        try:
            # PMI
            pmi_ = math.log(((p12) / (p1 * p2)), 2)
            npmi = pmi_ / - math.log(p12)

            # DICE
            dice_ = (p12 * 2) / (p1 + p2)

            #ZSCORE
            j = f12
            s = 24
            p = f2 / (totals[date] - f1)
            e = p * f1 * s
            z_score = (j - e) / math.sqrt(e * (1-p))
            return [pmi_,npmi,dice_,z_score]
        except Exception as e:
            print(e)
            return 0

In [10]:
totals = pd.read_csv(base_path + '/resources/totals/pl-total-tokens.csv')
totals = dict(zip(totals['month'],totals['n']))

In [72]:
df = data_loader.full('pl','lemmatized')

found 516 files in: /home/ruben/Documents/GitHub/ParlaMintCase/data/original/pl/pl-ana-txt/


In [73]:
dfw = utils.windowizer(df,words=["ekspert"],window=24,id_column="id",text_column="text")

In [74]:
words = [[w for w in t.split(' ')] for t in list(dfw['window'])]
words = [item for sublist in words for item in sublist]
words = dict(Counter(words))
words = dict({k:v for k,v in words.items() if v > 10 and len(k) > 3})

In [75]:
months = utils.month_generator("2019-05","2020-06")

pmi_data = []

for t in tqdm(list(words.keys())):
    for y in months:
        s = get_pmi_score(dfw,"ekspert",t,y)
        if s:
            pmi_data.append([t,y] + s)

100%|██████████| 956/956 [17:03<00:00,  1.07s/it]


In [82]:
pmi_data = pd.DataFrame(pmi_data,columns=['t','y','pmi','npmi','dice','zscore']).fillna(0)

In [83]:
def plot_(words,measure,rolling=True,rolling_window=4):
    d = pmi_data[pmi_data['t'].isin(words)]
    d = d[['t','y',measure]].reset_index(drop=True)
    d[measure] = d[measure].astype(float)
    d = d.pivot(index='y',columns='t',values=measure).reset_index().fillna(0.0)
    if rolling == True:
        for c in list(d.columns)[1:]:
            d[c] = d[c].rolling(rolling_window, win_type='gaussian').sum(std=3)
    fig, ax = plt.subplots()
    d.plot(x='y',y=words,figsize=(20,10),marker="o",ax=plt.gca(),cmap='Paired')
    ax.legend(title="Parties",fontsize=20)
    return d

In [84]:
list_words = list(set(pmi_data['t']))

In [79]:
#plot_(words=list_words,measure='dice',rolling=False)

In [85]:
d = pmi_data[['t','y','dice']].reset_index(drop=True)
d['dice'] = d['dice'].astype(float)
d = d.pivot(index='t',columns='y',values='dice').reset_index().fillna(0.0)

In [86]:
for y in list(d.columns)[1:]:
    print(y, ", ".join(list(d.sort_values(y,ascending=False)['t'])[:25]))
    print('')

2019-05 ekspert., ekspert, strategia, uczestniczyć, pośpiech, początek, towarzystwo, problematyka, opinia., opinia, zespół, konsultować, sejmowy, uwzględnić, pacjent, medium, medium., powołać, przygotowywać, spotkanie, wskazywać, naprawdę, wprost, konsultant, ekspercki

2019-06 ekspert, ekspert., szef, zespół, rzecznik, prawnik, wyjaśnić, rekomendacja, choroba, niezależny, legislacyjny., legislacyjny, biuro, polityk, polityk., senacki, towarzystwo, wraz, stowarzyszenie, według, rodzic, zajmować, przedstawiciel, uważać, konstytucja

2019-07 ekspert, ekspert., zewnętrzny, prawnik, raport, szpital, branżowy, szacować, uczelnia, standard, osiągnąć, uczestniczyć, naruszać, specjalista, potwierdzić, współpraca, międzynarodowy, sędzia, konsultować, konsultant, temat, dziedzina., dziedzina, profesor, kontrola

2019-08 ekspert., ekspert, ekspertyza, do_spraw, konferencja, dziennikarz, zaprosić, specjalista, apelować, medium., nauka, zespół, medium, choroba, szansa, produkcja, zapewniać, przeciw